In [42]:
import pandas as pd
from datetime import date
import re
pd.set_option('display.max_columns', None) #настройка, чтобы видеть все колонки

In [43]:
observations_path = 'data/observations-285432.csv'
# start_date = date(2021,10,2)
start_date = 'min'
# finish_date = date(2019,5,30)
finish_date = 'max'

In [44]:
def prepare_df(observations_path, start_date, finish_date):
    
    df_full = pd.read_csv(observations_path) #создаём новый огромный датафрейм из csv
    df = df_full[['id','user_login','created_at','quality_grade','scientific_name','common_name']].copy() #создаём новый маленький датафрейм с нужными столбцами
    del(df_full)
    df['created_at'] = pd.to_datetime(df['created_at']).dt.date #конвертируем столбец с датами наблюдений в тип datatime
    start_date = min(df['created_at']) if start_date == 'min' else start_date
    finish_date = max(df['created_at']) if finish_date == 'max' else finish_date

    return df, start_date, finish_date

In [45]:
def prepare_to_date(df, date_to):

    #prepare_to_date нужна чтобы создать список "лучших" 
    #наблюдателей с большим количеством наблюдений на определённую дату
    

    stat = pd.DataFrame()                                                               #создаём новый датафрейм
    counts = df[df['created_at']<=date_to].loc[:,'user_login'].value_counts()           #подсчитываем количество всех наблюдений у каждого пользователя с датой предшествующей или равной date_to
    stat['user'] = counts.index                                                         #создаём колонку с именами пользователей
    stat['obs_count'] = counts.to_list()                                                #создаём колонку с кол-вом всех наблюдений
    research = df[(df['quality_grade']=='research')&(df['created_at']<=date_to)].loc[:,'user_login'].value_counts()         #подсчитываем кол-во наблюдений со статусом research у каждого пользователя
    del(df)
    stat = stat.join(research,on='user', how='left')                                    #создаём колонку с кол-вом наблюдений research, объединяя список всех наблюдений со списком наблюдений research
    del(research)
    stat.rename(columns={'user':'user',                                                 #просто нужно переименовать один столбец из-за особенности join
                        'obs_count':'obs_count',
                        'user_login':'obs_res_count'},
                         inplace=True)
    stat.fillna(0,inplace=True)                                                         #заполняем нулями значения NaN (для тех у кого нет ни одного research)
    stat['obs_res_count'] = stat['obs_res_count'].astype('Int64')                       #меняем "х.0" на "x"
    stat.sort_values(by=['obs_count','obs_res_count','user'],ascending=False,inplace=True)
    stat.insert(1,'position','')
    stat.insert(2,'pos_cool','')
    stat['position'] = range(1,counts.shape[0]+1)                                       #создаём колонку с позицией, взяв кол-во пользователей из shape
    stat['pos_cool'] = stat[['obs_count','obs_res_count']].ne(stat[['obs_count','obs_res_count']].shift()).any(axis=1).cumsum()
    
    return stat

In [46]:
def shiftpos(shiftrow):
    
    if shiftrow['position_y'] == 0:
        cell = '+new'
    elif shiftrow['pos_cool_shift'] > 0:
        cell = '+' + str(shiftrow['pos_cool'])
    else: cell = ''

    return cell

def shiftplus(shift):

    cell = ('+' + str(shift)) if shift > 0 else ''

    return cell

In [47]:
def prepare_changes(df, start_date, finish_date):
    
    
    start_obs = prepare_to_date(df, start_date)
    finish_obs = prepare_to_date(df, finish_date)
    stat = finish_obs.merge(right=start_obs,how='left',left_on='user', right_on='user')   
    stat_changes = pd.DataFrame()
    stat_changes['position'] = stat['position_x']
    stat_changes['position_y'] = stat['position_y']
    stat_changes['pos_cool'] = stat['pos_cool_x']
    stat_changes['pos_cool_shift'] = stat['pos_cool_y'] - stat['pos_cool_x']
    stat_changes['user'] = stat['user']
    stat_changes['obs_count'] = stat['obs_count_x']
    stat_changes['obs_count_shift'] = stat['obs_count_x'] - stat['obs_count_y']
    stat_changes['obs_res_count'] = stat['obs_res_count_x']
    stat_changes['obs_res_count_shift'] = stat['obs_res_count_x'] - stat['obs_res_count_y'] 
    stat_changes.fillna(0,inplace=True)  
    stat_changes['pos_cool_shift'] = stat_changes['pos_cool_shift'].astype('Int64')                       #меняем "х.0" на "x"    
    stat_changes['obs_count_shift'] = stat_changes['obs_count_shift'].astype('Int64')                       #меняем "х.0" на "x"     
    stat_changes['obs_res_count_shift'] = stat_changes['obs_res_count_shift'].astype('Int64')                       #меняем "х.0" на "x"     
    stat_changes['pos_cool_shift'] = stat_changes.apply(shiftpos,axis=1)
    stat_changes['obs_count_shift'] = stat_changes['obs_count_shift'].apply(shiftplus)
    stat_changes['obs_res_count_shift'] = stat_changes['obs_res_count_shift'].apply(shiftplus)
    stat_changes.sort_values(by=['obs_count','obs_res_count','user'],ascending=False,inplace=True)

    return stat_changes

In [48]:
def changes_beauty(changes):

    changes = changes[((changes['pos_cool'] == 11).cumsum() != 1)].copy()
    changes['pos_cool'] = changes['pos_cool'].astype(str) + changes['pos_cool_shift']
    changes['obs_count'] = changes['obs_count'].astype(str) + changes['obs_count_shift']
    changes['obs_res_count'] = changes['obs_res_count'].astype(str) + changes['obs_res_count_shift']
    changes.drop(['position','position_y','pos_cool_shift','obs_res_count_shift','obs_count_shift'], axis=1, inplace=True)
    changes.columns = ['#','Кто','Наблюдения','Наблюдения,<br>    исследовательский уровень']
    
    return changes

In [49]:
def changes_html(changes):
  
  changes.to_html('output/changes.html', header=True, index=False, escape=False, justify='center', border=None)

  with open('output/changes.html', 'r') as file :
    filedata = file.read()

  filedata = filedata.replace(' class="dataframe"', '') # Replace the target string
  filedata = filedata.replace('<th>', '<th  style="vertical-align:top">')
  filedata = filedata.replace('+new', '<b style="font-size:62%;color:green">&nbsp;&nbsp;NEW</b>')
  filedata = re.sub('\+([0-9]+)',r'<b style="font-size:62%;color:green">&nbsp;&nbsp;&#8593;\1</b>',filedata)

  with open('output/changes.html', 'w') as file: # Write the file out again
    file.write(filedata)

In [50]:
obs, start_date, finish_date = prepare_df(observations_path, start_date, finish_date)
print(start_date, finish_date)
obs

2018-09-24 2022-12-23


,id,user_login,created_at,quality_grade,scientific_name,common_name
0,16876317,entomokot,2018-09-24,needs_id,Distantes,Сморчок высокий
1,16876320,entomokot,2018-09-24,needs_id,Attulus floricola,NaN
2,25788653,bio_4you,2019-05-26,needs_id,Malachiinae,Малашки
3,25928353,bio_4you,2019-05-28,research,Alnus glutinosa,Ольха чёрная
4,25928360,bio_4you,2019-05-28,research,Veronica chamaedrys,Вероника дубравная
...,...,...,...,...,...,...
930,144411345,budetinetakoe,2022-12-15,needs_id,Mammalia,Млекопитающие
931,144415529,budetinetakoe,2022-12-15,needs_id,Mammalia,Млекопитающие
932,144416151,budetinetakoe,2022-12-15,needs_id,Mammalia,Млекопитающие
933,144416393,budetinetakoe,2022-12-15,needs_id,Mammalia,Млекопитающие


In [51]:
changes = prepare_changes(obs, start_date, finish_date)
changes

,position,position_y,pos_cool,pos_cool_shift,user,obs_count,obs_count_shift,obs_res_count,obs_res_count_shift
0,1,0.0,1,+new,budetinetakoe,287,,192,
1,2,0.0,2,+new,anastasiiamerkulova,222,,160,
2,3,0.0,3,+new,elenasuslova,93,,88,
3,4,0.0,4,+new,mitrula_paludosa,62,,55,
4,5,0.0,5,+new,daria_kalinina,41,,37,
5,6,0.0,6,+new,taimyr,37,,37,
6,7,0.0,7,+new,kirill_kuptsov,37,,25,
7,8,0.0,8,+new,bio_4you,30,,23,
8,9,0.0,9,+new,elizaveta_boldinova,24,,24,
9,10,0.0,10,+new,naturalist40722,14,,13,


In [52]:
changes = changes_beauty(changes)
changes

,#,Кто,Наблюдения,"Наблюдения,<br> исследовательский уровень"
0,1+new,budetinetakoe,287,192
1,2+new,anastasiiamerkulova,222,160
2,3+new,elenasuslova,93,88
3,4+new,mitrula_paludosa,62,55
4,5+new,daria_kalinina,41,37
5,6+new,taimyr,37,37
6,7+new,kirill_kuptsov,37,25
7,8+new,bio_4you,30,23
8,9+new,elizaveta_boldinova,24,24
9,10+new,naturalist40722,14,13


In [53]:
changes_html(changes)